# Fast Training on full dataset
11th Apr

Same script as 5th Apr but I fixed the tensorboard logging so it should log sensibly now.

Model trains fine on tiny dataset - inspect initial training on full dataset with large batch size to see if it just blows up very early on. Turned off validation split to increase the logging speed.

In [ ]:
import os
from dock2hit.utils import write_slurm_script

current_dir = os.getcwd()
data_dir = '/rds-d2/user/wjm41/hpc-work/datasets/Ugis/datasets'
log_dir = '/rds-d2/user/wjm41/hpc-work/datasets/Ugis/runs/ultra-large/D4-debug'

dataset='D4'
model_name = 'quick_train_ultra'
model_dir = f'/rds-d2/user/wjm41/hpc-work/models/dock2hit/{dataset}'

model_path = f'{model_dir}/{model_name}'
# data_path = f'{data_dir}/{dataset}_test.csv'
data_path = f'{data_dir}/ultra_large/{dataset}_docked_valid.pkl'

script = f'python -c "from dock2hit.train_and_validate.multithread import main; main()"'

log_step = 10
args = [
    f'-path_to_train_data {data_path}',
    '-batch_size 256',
    '-minibatch_size 64',
    '-n_epochs 10000',
    f'-save_dir {model_path}',
    f'-log_dir {log_dir}',
    f'-steps_per_log {log_step}'
    ]

file_name = 'subm_quick_train_ultra'
run_time = '0:15:00'
output_name = f'{current_dir}/subm_quick_train_ultra.out'

write_slurm_script(job_name=f'{dataset}_quick_train_ultra',
                   run_time=f'{run_time}',
                   output_name=output_name,
                   package_dir='/rds-d2/user/wjm41/hpc-work/datasets/Ugis/',
                   script=script,
                   args=args,
                   file_name=file_name,
                   email=True
                    )

print(f"Submitted quick_trainging jobs for {model_path} on {data_path}")

!sbatch {file_name}

In [ ]:
!squeue -u wjm41


In [ ]:
!scancel 58464108


Hypothesis that bad y-scaling from datapoints with y>0 is causing low gradients: cutting those datapoints here

In [ ]:
import pandas as pd

df = pd.read_pickle('/rds-d2/user/wjm41/hpc-work/datasets/Ugis/datasets/ultra_large/D4_docked_valid.pkl')

df_without_positive_y = df.query('dockscore < 0')
df_without_positive_y.to_pickle(
    '/rds-d2/user/wjm41/hpc-work/datasets/Ugis/datasets/ultra_large/D4_negative.pkl')


In [ ]:
df.describe()


Submit job with only y<0

In [1]:
import os
from dock2hit.utils import write_slurm_script

current_dir = os.getcwd()
data_dir = '/rds-d2/user/wjm41/hpc-work/datasets/Ugis/datasets'
log_dir = '/rds-d2/user/wjm41/hpc-work/datasets/Ugis/runs/ultra-large/D4-debug'

dataset = 'D4'
model_name = 'quick_train_ultra_negative'
model_dir = f'/rds-d2/user/wjm41/hpc-work/models/dock2hit/{dataset}'

model_path = f'{model_dir}/{model_name}'

data_path = f'{data_dir}/ultra_large/{dataset}_negative.pkl'

script = f'python -c "from dock2hit.train_and_validate.multithread import main; main()"'

log_step = 10
args = [
    f'-path_to_train_data {data_path}',
    '-batch_size 256',
    '-minibatch_size 64',
    '-lr 1e-5',
    '-n_epochs 10000',
    f'-save_dir {model_path}',
    f'-log_dir {log_dir}',
    f'-steps_per_log {log_step}'
]

file_name = 'subm_quick_train_ultra_negative'
run_time = '0:15:00'
output_name = f'{current_dir}/{file_name}.out'

write_slurm_script(job_name=f'{file_name}',
                   run_time=f'{run_time}',
                   output_name=output_name,
                   package_dir='/rds-d2/user/wjm41/hpc-work/datasets/Ugis/',
                   script=script,
                   args=args,
                   file_name=file_name,
                   email=True
                   )

print(f"Submitted quick_trainging jobs for {model_path} on {data_path}")

!sbatch {file_name}
